# Project 1: Advanced Image Classification with CNNs & Transfer Learning

**Objective:** Classify images from the Intel Image Classification dataset using a custom CNN and a pre-trained model with transfer learning.

## 1. Setup and Imports

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Note: To run this, first download and unzip the dataset from Kaggle:
# https://www.kaggle.com/datasets/puneet6060/intel-image-classification

## 2. Load and Prepare the Dataset

In [4]:
# Define paths (Update these paths to where you've stored the data)
base_dir = './intel-image-classification'
train_dir = os.path.join(base_dir, 'seg_train', 'seg_train')
val_dir = os.path.join(base_dir, 'seg_test', 'seg_test') # Using test set as validation for simplicity

# Define parameters
IMG_SIZE = 150
BATCH_SIZE = 32

# Load datasets using tf.keras utility
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE)
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    shuffle=False, # No need to shuffle validation data
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE, IMG_SIZE)
)

class_names = train_dataset.class_names
print(f"Found classes: {class_names}")

# Prefetching for performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.
Found classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


## 3. Data Augmentation and Rescaling

In [5]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

rescale = layers.Rescaling(1./255)

## 4. Model 1: CNN from Scratch

In [6]:
def build_scratch_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = rescale(x)
    
    x = layers.Conv2D(32, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model

scratch_model = build_scratch_model((IMG_SIZE, IMG_SIZE, 3), len(class_names))
scratch_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

scratch_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,735,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,829,126 (18.42 MB)

 Trainable params: 4,829,126 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
print("Training the scratch model...")
history_scratch = scratch_model.fit(
    train_dataset,
    epochs=25, # A few epochs to show the concept
    validation_data=validation_dataset,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

Training the scratch model...
Epoch 1/25
181/439 ━━━━━━━━━━━━━━━━━━━━ 49s 193ms/step - accuracy: 0.3306 - loss: 1.6397

KeyboardInterrupt: 

## 5. Model 2: Transfer Learning with EfficientNetB0

In [ ]:
def build_transfer_model(input_shape, num_classes):
    base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False # Freeze the base
    
    inputs = layers.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = rescale(x) # EfficientNet has its own internal rescaling, but this is fine.
    x = base_model(x, training=False) # Important to set training=False
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs, outputs)
    return model, base_model

transfer_model, base_model = build_transfer_model((IMG_SIZE, IMG_SIZE, 3), len(class_names))
transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

transfer_model.summary()

In [ ]:
print("Training with transfer learning (feature extraction)...")
history_transfer = transfer_model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

### Fine-Tuning the Transfer Learning Model
Now we unfreeze some of the top layers of the base model and train with a very low learning rate.

In [ ]:
base_model.trainable = True

# Freeze all layers except the top few
for layer in base_model.layers[:-20]: # Fine-tune the last 20 layers
    layer.trainable = False

transfer_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), # Very low learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Fine-tuning the transfer learning model...")
history_fine_tune = transfer_model.fit(
    train_dataset,
    epochs=10,
    initial_epoch=history_transfer.epoch[-1], # Continue from where we left off
    validation_data=validation_dataset,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

## 6. Model Evaluation and Comparison

In [ ]:
def plot_history(history, title):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.suptitle(title, fontsize=16)
    plt.show()

plot_history(history_scratch, 'Scratch Model Performance')
# Combine transfer and fine-tune histories for a full plot
full_transfer_history_acc = history_transfer.history['accuracy'] + history_fine_tune.history['accuracy']
full_transfer_history_val_acc = history_transfer.history['val_accuracy'] + history_fine_tune.history['val_accuracy']
full_transfer_history_loss = history_transfer.history['loss'] + history_fine_tune.history['loss']
full_transfer_history_val_loss = history_transfer.history['val_loss'] + history_fine_tune.history['val_loss']

plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(full_transfer_history_acc, label='Training Accuracy')
plt.plot(full_transfer_history_val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(full_transfer_history_loss, label='Training Loss')
plt.plot(full_transfer_history_val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.suptitle('Transfer Learning + Fine-Tuning Performance', fontsize=16)
plt.show()

In [ ]:
def evaluate_model(model, dataset, model_name):
    print(f"\n--- Evaluating {model_name} ---")
    loss, accuracy = model.evaluate(dataset)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    
    # Get predictions and true labels
    y_pred_probs = model.predict(dataset)
    y_pred = np.argmax(y_pred_probs, axis=1)
    y_true = np.concatenate([y for x, y in dataset], axis=0)
    
    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

evaluate_model(scratch_model, validation_dataset, 'CNN from Scratch')
evaluate_model(transfer_model, validation_dataset, 'Transfer Learning (Fine-Tuned)')

## 7. Conclusion

As seen from the evaluation metrics and plots, the transfer learning approach with fine-tuning significantly outperforms the CNN built from scratch. This demonstrates the immense value of leveraging pre-trained models for computer vision tasks, as they provide a powerful foundation of learned features that can be adapted to new, specific problems.